# ⚙️ Optimal Power Flow primer z različnimi scenariji
Uporaba pandapower za izračun pretoka moči, stroškov in obremenitev transformatorja z različnimi vhodnimi pogoji.

In [1]:
import pandapower as pp
import numpy as np

In [2]:
net = pp.create_empty_network()

In [3]:
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

In [4]:
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

0

In [5]:
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

2

In [6]:
pp.create_load(net, bus2, p_mw=60, controllable=False)
pp.create_load(net, bus3, p_mw=70, controllable=False)
pp.create_load(net, bus4, p_mw=10, controllable=False)

2

In [7]:
eg = pp.create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)
costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

In [11]:
pp.runopp(net, delta=1e-16)
print('Analiza osnovnega primera')
print(f"Moč toge mreže \n{net.res_ext_grid}")
print(f"Moč generatorjev \n{net.res_gen}")
print(f"Skupen strošek: \n{net.res_cost} €")

Analiza osnovnega primera
Moč toge mreže 
        p_mw    q_mvar
0  49.953012 -2.147126
Moč generatorjev 
        p_mw    q_mvar  va_degree     vm_pu
0   0.000598  2.992989  -6.232710  0.985230
1  93.304317  3.453173  -1.237784  1.025709
Skupen strošek: 
1619.1908981410588 €


In [8]:
net.poly_cost.cp1_eur_per_mw.at[costeg] = 10
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen2] = 12
pp.runopp(net, delta=1e-16)
print('\nAnaliza drugega primera')
print(f"Moč toge mreže \n{net.res_ext_grid}")
print(f"Moč generatorjev \n{net.res_gen}")
print(f"Skupen strošek: \n{net.res_cost} €")
print(f"Obremenitev transformatorja: \n{net.res_trafo.loading_percent}")

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



Analiza drugega primera
Moč toge mreže 
         p_mw    q_mvar
0  144.559166  9.193021
Moč generatorjev 
       p_mw     q_mvar  va_degree     vm_pu
0  0.000079   8.601766 -16.426835  0.967619
1  0.000225  10.594623 -13.481007  0.989756
Skupen strošek: 
1445.595544859482 €
Obremenitev transformatorja: 
0    144.851179
Name: loading_percent, dtype: float64


In [9]:
net.trafo["max_loading_percent"] = 50
pp.runopp(net, delta=1e-16)
print('\nAnaliza tretjega primera')
print(f"Moč toge mreže \n{net.res_ext_grid}")
print(f"Moč generatorjev \n{net.res_gen}")
print(f"Skupen strošek: \n{net.res_cost} €")
print(f"Obremenitev transformatorja: \n{net.res_trafo.loading_percent}")

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



Analiza tretjega primera
Moč toge mreže 
        p_mw    q_mvar
0  49.953012 -2.147126
Moč generatorjev 
        p_mw    q_mvar  va_degree     vm_pu
0   0.000598  2.992989  -6.232710  0.985230
1  93.304317  3.453173  -1.237784  1.025709
Skupen strošek: 
1619.1908981410575 €
Obremenitev transformatorja: 
0    49.999136
Name: loading_percent, dtype: float64
